# The explanation of checking for vanishing gradient descent problem explanation is given in model training part.

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(3)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,1,1,101348.88,1
1,608,41,1,83807.86,1,0,1,112542.58,0
2,502,42,8,159660.80,3,1,0,113931.57,1


In [3]:
dataset.isnull().sum()

CreditScore        0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
input_data = dataset.iloc[:,:-1]
output_data = dataset['Exited']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(input_data,output_data,test_size=0.2, random_state=10)

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
ss = StandardScaler()
input_data = pd.DataFrame(ss.fit_transform(input_data), columns=input_data.columns)

In [9]:
df = pd.DataFrame(input_data)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,0.293517,-1.041760,-1.225848,-0.911583,0.646092,0.970243,0.021886
1,-0.440036,0.198164,-1.387538,0.117350,-0.911583,-1.547768,0.970243,0.216534
2,-1.536794,0.293517,1.032908,1.333053,2.527057,0.646092,-1.030670,0.240687
3,0.501521,0.007457,-1.387538,-1.225848,0.807737,-1.547768,-1.030670,-0.108918
4,2.063884,0.388871,-1.041760,0.785728,-0.911583,0.646092,0.970243,-0.365276


In [10]:
import tensorflow 

In [11]:
from keras.layers import Dense, BatchNormalization, Dropout
from keras.regularizers import L2
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [12]:
ann = Sequential()

In [13]:
# to understand input node
input_data.shape

(10000, 8)

In [14]:
ann.add(Dense(6,input_dim = 8, activation='relu',kernel_regularizer=L2(l2=0.01)))#for regularization we can use L1, L2, L1&L2
ann.add(Dense(4, activation='relu',kernel_regularizer=L2(l2=0.01)))
ann.add(Dense(2, activation='relu',kernel_regularizer=L2(l2=0.01)))
ann.add(Dropout(0.3))
ann.add(Dense(1, activation='sigmoid')) #use of sigmoid activation function because our output is in binary format

C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
ann.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# for batch size
x_train.shape

(8000, 8)

In [24]:
ann.fit(x_train, y_train, batch_size=200, epochs = 40,validation_data=(x_test,y_test),callbacks=EarlyStopping())
# The drop of the loss in this model training step by step proves there doesnt exist vanishing gradient problem.

Epoch 1/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8010 - loss: 0.6297 - val_accuracy: 0.7890 - val_loss: 0.6246
Epoch 2/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8001 - loss: 0.6284 - val_accuracy: 0.7890 - val_loss: 0.6202
Epoch 3/40
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7984 - loss: 0.6272 - val_accuracy: 0.7890 - val_loss: 0.6237


In [25]:
train_accuracy = ann.history.history['accuracy']
test_accuracy = ann.history.history['val_accuracy']#this is only training accuracy but after validation_data given in model, val_accuracy and val_loss obtained

In [26]:
prd = ann.predict(x_test)
prd_data = []
for i in prd:
    if i[0] > 0.5:
        prd_data.append(1)
    else:
        prd_data.append(0)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [27]:
prd1 = ann.predict(x_train)
prd_data1 = []
for i in prd1:
    if i[0] > 0.5:
        prd_data1.append(1)
    else:
        prd_data1.append(0)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(y_test, prd_data) * 100

78.9

In [30]:
accuracy_score(y_train, prd_data1) * 100 # the model is overfitting

79.8125